<a href="https://colab.research.google.com/github/jackiekuen2/notes-handson-ml-tf/blob/master/ch6_ExerciseQ7Q8_DecisionTreeMoons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q7. Decision Tree for moon dataset

In [0]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=42)

params = {
    'max_leaf_nodes': list(range(2, 100)),
    'min_samples_split': list(range(2, 5))
}

grid_search_cv = GridSearchCV(tree_clf, param_grid=params, n_jobs=-1, verbose=2, cv=5)

grid_search_cv.fit(X_train, y_train)

Fitting 5 folds for each of 294 candidates, totalling 1470 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 1470 out of 1470 | elapsed:   14.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
  

In [0]:
grid_search_cv.best_score_

0.85925

In [0]:
grid_search_cv.best_params_

{'max_leaf_nodes': 23, 'min_samples_split': 2}

In [0]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.8735

# Q8. Building own Random Forest

In [0]:
# Split training set inti 1000 subsets
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train)-n_instances, random_state=42)

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [0]:
# Clone the above best estimator (model) for 1000 times (i.e. n_trees) for training
from sklearn.base import clone

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

In [0]:
# To average out an accuracy score over 1000 trees
import numpy as np

np.mean(accuracy_scores)

0.7988824999999999

The average accuracy of 1000 Decision Trees is 79.89%

In [0]:
# Initialize the final prediction as empty array
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

In [0]:
Y_pred.shape

(1000, 2000)

In [0]:
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [0]:
from scipy.stats import mode
y_pred_majority_vote, n_votes = mode(Y_pred, axis=0)

In [0]:
accuracy_score(y_test, y_pred_majority_vote.reshape(-1))

0.8735

The accuracy of majority-vote predictions is 87.35% (i.e. Random Forest)